In [1]:
TREE_CHOICES = {
    "Abdomen": {"Liver": ["Raw", "Contrast"], "Pancreas": ["Raw", "Contrast"]},
    "Arterial": {"Carotids": ["CIMT120", "CIMT150", "LongAxis", "Mixed", "ShortAxis"]},
    "Brain": {
        "MRI": [
            "SagittalRaw",
            "SagittalReference",
            "CoronalRaw",
            "CoronalReference",
            "TransverseRaw",
            "TransverseReference",
        ]
    },
    "Eyes": {"Fundus": ["Raw"], "OCT": ["Raw"]},
    "Heart": {
        "MRI": [
            "2chambersRaw",
            "2chambersContrast",
            "3chambersRaw",
            "3chambersContrast",
            "4chambersRaw",
            "4chambersContrast",
        ]
    },
    "Musculoskeletal": {
        "FullBody": ["Figure", "Flesh", "Mixed", "Skeleton"],
        "Knees": ["DXA"],
        "Hips": ["DXA"],
        "Spine": ["Coronal", "Sagittal"],
    },
    "PhysicalActivity": {
        "FullWeek": [
            "GramianAngularField1minDifference",
            "GramianAngularField1minSummation",
            "MarkovTransitionField1min",
            "RecurrencePlots1min",
        ]
    },
}

In [7]:
dims = ""
for dim in TREE_CHOICES.keys():
    dims += f'"{dim}" '
dims

'"Abdomen" "Arterial" "Brain" "Eyes" "Heart" "Musculoskeletal" "PhysicalActivity" '